In [1]:
import pandas as pd
import time
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box
import matplotlib.pyplot as plt
    
def read_sort_dict(even_dict1, odd_dict1):
        """Вывод отсортированнного словаря """

        list_key = list(even_dict1.keys())
        list_key.sort()
        print('Статистика четного вхождения:')
        chance = 0
        for key in list_key:
            part = round(even_dict1[key]/sum(even_dict1.values()), 2)
            chance += even_dict1[key]/sum(even_dict1.values()) 
            print(key, 'подряд - ', even_dict1[key], 'раз ', part, ' --> ', round(chance, 2))
        print('')    
        print('Статистика нечетного вхождения:')
        chance = 0

        list_key = list(odd_dict1.keys())
        list_key.sort()
        for key in list_key:
            part = round(odd_dict1[key]/sum(odd_dict1.values()), 2)
            chance += odd_dict1[key]/sum(odd_dict1.values()) 
            print(key, 'подряд - ', odd_dict1[key], 'раз ', part, ' --> ', round(chance, 2))
            
def create_df(list_df):
    """Формирование дата фрема команды из таблицы регулярки и плейофф(если есть)"""
    
    df_poff = pd.DataFrame()
    for df in list_df:
        df = df[['Date', 'Unnamed: 5', 'Opponent', 'Tm', 'Opp', 'Streak']].rename(columns={'Unnamed: 5':'Road'})
        df = df[(df['Tm'] != 'Tm')]
        df.dropna(subset=['Tm'], inplace=True) # Remove missing values.
        df.reset_index(drop=True, inplace=True)
        df['Date'] = pd.to_datetime(df['Date'])
        df_regul = df_poff
        df_poff = df

    df_total = pd.DataFrame()
    df_total = df_regul.append(df_poff, ignore_index=True)
    df_total.sort_values('Date', ascending=False, inplace=True)
    df_total['Tm'] = df_total.Tm.astype('int32')
    df_total['Opp'] = df_total.Opp.astype('int32')
    
    return df_total

def create_dicts(df_total):
    ''' Раздел счетчика чет нечет всего '''

    even = 0 # четный
    odd = 0 # нечетный

    even_dict = {}
    odd_dict = {}
    sum_even = 0
    sum_odd = 0
    list_ev_odd = []
    for it in df_total.Tm:
        if int(it) % 2 == 0:
            sum_even += 1 # Общая четных
            if odd != 0 and str(odd) not in odd_dict:
                odd_dict.setdefault(str(odd), 1)
                list_ev_odd.append(odd*-1)
            elif odd != 0 and str(odd) in odd_dict:
                odd_dict[str(odd)] += 1
                list_ev_odd.append(odd*-1)
            odd = 0
            even += 1
        elif int(it) % 2 != 0:
            sum_odd += 1 # Общая нечетных
            if even != 0 and str(even) not in even_dict:
                even_dict.setdefault(str(even), 1)
                list_ev_odd.append(even)
            elif even != 0 and str(even) in even_dict:
                even_dict[str(even)] += 1
                list_ev_odd.append(even)
            even = 0
            odd += 1

   
    if odd != 0 and str(odd) not in odd_dict: # т.к. запись в словарь происходит после смены чет/нечет то по окончанию
        odd_dict.setdefault(str(odd), 1)      # цикла переменная накопилась но смены уже не произойдет, поэтому записываю
        list_ev_odd.append(odd*-1)            # переменную после выхода из цикла
    elif odd != 0 and str(odd) in odd_dict:  
        odd_dict[str(odd)] += 1
        list_ev_odd.append(odd*-1)
    if even != 0 and str(even) not in even_dict:
        even_dict.setdefault(str(even), 1)
        list_ev_odd.append(even)
    elif even != 0 and str(even) in even_dict:
        even_dict[str(even)] += 1
        list_ev_odd.append(even)
   
    return sum_even, sum_odd, even_dict, odd_dict, list_ev_odd

teams_west = {'DAL':'Dallas Mavericks', 'DEN':'Denver Nuggets', 'GSW':'Golden State Warriors', 
              'HOU':'Houston Rockets', 'LAC':'LA Clippers', 'LAL':'LA Lakers', 'MEM':'Memphis Grizzlies',
              'MIN':'Minnesota Timberwolves', 'NOP':'New Orleans Pelicans', 'OKC':'Oklahoma City Thunder', 
              'PHO':'Phoenix Suns', 'POR':'Portland Trail Blazers', 'SAC':'Sacramento Kings', 'SAS':'San Antonio Spurs',
              'UTA':'Utah Jazz'
             }
#Абревиатура Charlotte Hornets, на самом деле CHA, но на сайте-источнике CHO
teams_east = {'ATL':'Atlanta Hawks', 'BOS':'Boston Celtics', 'BRK':'Brooklyn Nets', 'CHO':'Charlotte Hornets',
              'CHI':'Chicago Bulls', 'CLE':'Cleveland Cavaliers', 'DET':'Detroit Pistons', 'IND':'Indiana Pacers', 
              'MIA':'Miami Heat', 'MIL':'Milwaukee Bucks', 'NYK':'New York Knicks', 'ORL':'Orlando Magic', 
              'PHI':'Philadelphia 76ers', 'TOR':'Toronto Raptors', 'WAS':'Washington Wizards'
             }

full_list_teams = {**teams_west, **teams_east}
def create_checklist(teams):
    selected_data = []

    names = []
    checkbox_objects = []
    for key, name in teams.items():
        checkbox_objects.append(widgets.ToggleButton(description=key, value=False, tooltip=name))        
        names.append(key)

    arg_dict = {names[i]: checkbox for i, checkbox in enumerate(checkbox_objects)}
    
    ui = widgets.HBox(children=checkbox_objects)

    def select_data(**kwargs):
        selected_data.clear()

        for key in kwargs:
            if kwargs[key] is True:
                selected_data.append(key)

    out = widgets.interactive_output(select_data, arg_dict)
    display(ui)
    return out, selected_data

In [2]:
list_teams_west = create_checklist(teams_west)
list_teams_east = create_checklist(teams_east)

In [ ]:
#EMPY CELL, don't to delete

In [13]:
start_link = 'https://www.basketball-reference.com/teams/'
end_link = '/2021_games.html'
count = 0
selected_teams = list_teams_west[1] + list_teams_east[1]
all_ = len(selected_teams)
N = 3 # Поиск длины серии не менее N
stat_all_teams = False # Отображать стату для всех выбранных команд, а не только команды с длиной серии >= N

if selected_teams:
    print('EvOdd NBA', time.strftime("%d-%m-%Y, %H:%M:%S") )
else:
    print('Не выбрана ни одна команда')

for team in selected_teams:
    count += 1 
    link = start_link + team + end_link
    time.sleep(0.3)
    list_df = pd.read_html(link)
    print(f'#############################')
    print(f'----- команда {count} из {all_} -----')
    print(f'{full_list_teams[team].upper()}')
    print('')
    
################################################################################

    df_total = create_df(list_df) #Формирование дата фрема команды из таблицы регулярки и плейофф(если есть)

####################################################################    
    # Поиск первой серии чет\нечет в последних играх
    even_ = 0
    odd_ = 0

    for item in df_total.Tm:
        if int(item) % 2 == 0:
            if odd_ != 0 : 
                if odd_ >= N:
                    print(f'НЕЧЕТНО {odd_} раз(а) подряд', end='  ')
                else:
                    print(f'Не найдено серии из {N}-х подряд исходов')
                break
            odd_ = 0
            even_ += 1
        else:
            if even_ != 0 :
                if even_ >= N:
                    print(f'ЧЕТНО {even_} раз(а) подряд', end='  ')
                else:
                    print(f'Не найдено серии из {N}-х подряд  исходов')
                break
            even_ = 0
            odd_ += 1

####################################################################        
    if (even_ >= N or odd_ >= N) or stat_all_teams:
        sum_even, sum_odd, even_dict, odd_dict, list_ev_odd = create_dicts(df_total)      # Раздел счетчика чет нечет всего
        list_ev_odd.reverse()

        print('Четных:',sum_even, '(', round(sum_even/(sum_even+sum_odd),2), ')', end=' ')
        print(' Нечетных:',sum_odd, '(', round(sum_odd/(sum_even+sum_odd),2), ')')

        print('')
        read_sort_dict(even_dict, odd_dict) # Вывод отсортированнного словаря

#         display(df_total.head(5))
        pd.Series(list_ev_odd).plot.bar(figsize=(15,5)).grid(axis='y')
        plt.show()
print('')
print('Программа закончила работу'.upper())


EvOdd WNBA 17-10-2021, 14:11:02
#############################
----- команда 1 из 1 -----
ATLANTA HAWKS

Не найдено серии из 3-х подряд исходов

ПРОГРАММА ЗАКОНЧИЛА РАБОТУ


## Конец документа